## Library Preparation

In [ ]:
# Clone repository dan instalasi dependensi RapidsAI
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

# Instalasi library tambahan
!pip install app_store_scraper
!pip install openai

# Import library untuk clustering dan visualisasi
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from cuml.cluster import KMeans

# Import library untuk analisis data
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

# Import library untuk pemrosesan NLP
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Import library untuk scraping ulasan
from app_store_scraper import AppStore

# Mount Google Drive untuk penyimpanan file
from google.colab import drive
drive.mount('/content/drive')

# Menambahkan path ke dalam sistem
import sys
sys.path.insert(0, "/content/drive/MyDrive/Deep Learning/FCMeans")

# Import library tambahan untuk clustering
from fcmeans import fcmeans

# Import library tambahan untuk NLP
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import gensim.downloader

# Instalasi OpenAI
import openai

# Fungsi tambahan
import re
from google.colab import files


In [ ]:
!pip install app_store_scraper

In [ ]:
# Mengecek versi library yang terpasang
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
import seaborn as sns
import nltk
import gensim
import sklearn
import openai
import tqdm
import re
import sys
import app_store_scraper
from transformers import __version__ as transformers_version
# Output versi
versions = {
    "pandas": pd.__version__,
    "numpy": np.__version__,
    "tensorflow": tf.__version__,
    "matplotlib": matplotlib.__version__,
    "seaborn": sns.__version__,
    "nltk": nltk.__version__,
    "gensim": gensim.__version__,
    "sklearn": sklearn.__version__,
    "openai": openai.__version__,
    "tqdm": tqdm.__version__,
    "re": re.__version__,
    "transformers": transformers_version,
    "app_store_scraper": app_store_scraper.__version__,
}

# Menampilkan hasil
versions_df = pd.DataFrame(list(versions.items()), columns=["Library", "Version"])
print(versions_df)


## Data Scraping App Store

In [ ]:
# Inisialisasi objek AppStore untuk aplikasi Tokocrypto
data = AppStore(country='id', app_name='Tokocrypto: Trade BTC & Crypto', app_id='1538556690')

#Mengatur jumlah ulasan yang akan diambil
data.review(how_many=99999)

In [ ]:
data.review # Mengakses daftar ulasan

In [ ]:
data1 = pd.DataFrame(np.array(data.reviews),columns=['review']) # Membuat DataFrame dari ulasan
data2 = data1.join(pd.DataFrame(data1.pop('review').tolist())) # Membuat data ulasan menjadi kolom terstruktur

In [ ]:
data2 = data2.sort_values('date', ascending=False) # Mengurutkan ulasan berdasarkan tanggal (terbaru di atas)
data2.head()

In [ ]:
Tokocrypto = data2[['date','userName','review','rating']] # Memilih kolom yang akan digunakan
Tokocrypto

In [ ]:
Tokocrypto.to_excel('Tokocrypto.xlsx', index=False) # Menyimpan DataFrame sebagai file Excel tanpa indeks

## Data Cleaning

In [ ]:
df = pd.read_excel("data_tokocrypto.xlsx")
df

In [ ]:
## Import kamus untuk mengubah singkatan
kamus_alay = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
## Membuat dictionary untuk memetakan singkatan dari kata
nor_dict = {}
for index, row in kamus_alay.iterrows():
  if row[0] not in nor_dict:
    nor_dict[row[0]] = row[1]
def clean_text_1(text):
  # 01 Konversi ke huruf kecil semua
  text = text.strip().lower()
  # 02 Menghilangkan tautan www.* atau https?://*
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', text)
  # 03 Menghilangkan @username
  text = re.sub('@[^\s]+','', text)
  # 04 Menghilangkan # dari #kata
  text = re.sub(r'#([^\s]+)', r'\1', text)
  # 05 Menghilangkan tanda !, ?, +, &
  text = re.sub(r'[!&?+%]', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  # 07 Menghilangkan spasi berlebihan
  text = re.sub('[\s]+', ' ', text)
  text = re.sub(r'[ðÿ]', '', text)
  text = re.sub(r'[ðŸ‘]', '', text)
  text = re.sub(r'[ðŸ]', '', text)
  # 08 Menghilangkan emoticon dan semacamnya
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F" # emoticons
                              u"\U0001F300-\U0001F5FF" # symbols & pictographs
                              u"\U0001F680-\U0001F6FF" # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF" # flags (ios)
                              "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'', text)

  # Pembersihan kata
  words = text.split()
  tokens=[]
  for ww in words:
      # Memisahkan kata berulang
      for w in re.split(r'[-/\s]\s*', ww):
          # Menghapus huruf berulang yang lebih dari dua kali
          pattern = re.compile(r"(.)\1{1,}", re. DOTALL)
          w = pattern.sub(r"\1\1", w)
          w = w.strip('\'"?,.')
          # Memeriksa apakah suatu kata terbentuk dari minimal dua huruf
          #val = re.search(r"^[a-zA-Z][a-zA-Z][a-zA-Z]*$", w)
          if w in nor_dict:
              w = nor_dict[w]
          if w == "rt":
          #or val is None:
              continue
          else:
            tokens.append(w.lower())
  text = " ".join(tokens)
  return text.strip()

def clean_text_2(text):
    # Untuk CTFIDF
    # 01 Menghilangkan tanda baca
    text = re.sub(r' [^\w\s]','', text)
    # 02 Menghilangkan angka
    text = re.sub("[0-9]","", text)
    return text.strip()

## Untuk CTFIDF
additional_stopwords = ['gue', 'pas', 'banget', 'bikin', 'gua', 'ya', 'kalo', 'saya', 'nya', 'saat',
                        'sih', 'deh', 'dll', 'nih', 'donk', 'min', 'ny', 'si', 'eh', 'tu', 'mah',
                        'loh', 'aja', 'bagus','membantu', 'good', 'mantap', 'mudah', 'cepat', 'ðŸ‘', 'alhamdulillah', 'keren',
                        'terimakasih','kali','top','best','bad','useless','trash','job','worst']
stop_words = stopwords.words('indonesian') + additional_stopwords + stopwords.words('english')
stop_words = set(stop_words)

def remove_stop_words(s):
    return " ".join(word for word in s.split() if word not in stop_words)

In [ ]:
# Membuat kolom text_1 sebagai representasi string dari kolom ulasan
df["text_1"] = df["ulasan"].astype(str)

# Membersihkan teks di kolom text_1 dengan fungsi clean_text_1 dan clean_text_2
df["text_1"] = df["text_1"].map(lambda x: clean_text_1(x))
df["text_2"] = df["text_1"].map(lambda x: clean_text_2(x))

# Menghapus stopwords dari kolom text_2
df['text_2'] = df['text_2'].apply(lambda x: remove_stop_words(x))

# Memfilter baris berdasarkan panjang teks (lebih dari 3 kata)
df = df[df["text_1"].apply(lambda x: len(x.split()) > 3)]

# Memfilter baris yang tidak mengandung kata-kata tertentu
df = df[~df['text_1'].str.contains('game|hero|match|epep|player|ping|resolusi', case=False)]

# Menampilkan DataFrame
df

## Zero-Shot GPT

In [ ]:
#Memisahkan data untuk data testing
df_testing = df[df['label'].notna()]
df_testing

In [ ]:
def get_completion_from_message(message,
                                model="gpt-4o",
                                temperature=0):
    client = OpenAI(
        api_key='OPENAI_API_KEY',
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

def sentiment(ulasan):
    delimiter = "#####"
    system_message = f"""Saya sedang melakukan analisis sentimen terhadap ulasan pelanggan dari sebuah aplikasi \
    exchange crypto bernama tokocrypto
    Petunjuk:
    Jawab hanya dengan satu kata saja: "positif" atau "negatif"!"""

    system_message = {'role': 'system', 'content': system_message}

    # Menyiapkan pesan pengguna untuk ulasan yang ingin diuji
    user_message = [{'role': 'user', 'content': f"{delimiter}{ulasan}{delimiter}"}]

    # Gabungkan pesan sistem dan pesan pengguna
    messages = [system_message] + user_message

    # Dapatkan respons model
    response = get_completion_from_message(messages).strip()

    return response

# Tambahkan kolom 'sentimen' berdasarkan hasil dari fungsi sentiment
df_testing['zero-shot'] = df_testing['text_1'].apply(sentiment)

# Cetak dataframe dengan kolom sentimen yang baru
print(df_testing)


## One-Shot Positive

In [ ]:
import pandas as pd
from openai import OpenAI

def get_completion_from_message(message,
                                model="gpt-4o",
                                temperature=0):
    client = OpenAI(
        api_key='OPENAI_API_KEY',
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

def sentiment_single(ulasan):
    delimiter = "#####"

    # Menyediakan contoh one-shot positif
    example_input = "UI aplikasinya simple, jadi bikin gabingung buat orang yang baru belajar"
    example_output = "positif"

    system_message = f"""Saya sedang melakukan analisis sentimen terhadap ulasan pelanggan dari sebuah aplikasi \
    exchange crypto bernama tokocrypto.
    Berikut adalah contoh ulasan dan jawabannya:
    Ulasan: {example_input}
    Jawaban: {example_output}

    Petunjuk:
    - Jawab hanya dengan satu kata: "positif" atau "negatif" berdasarkan ulasan yang diberikan.
    - Ulasan: {delimiter}{ulasan}{delimiter}"""

    system_message = {'role': 'system', 'content': system_message}

    # Menyiapkan pesan pengguna untuk ulasan yang ingin diuji
    user_message = [{'role': 'user', 'content': f"{delimiter}{ulasan}{delimiter}"}]

    # Gabungkan pesan sistem dan pesan pengguna
    messages = [system_message] + user_message

    # Dapatkan respons model
    response = get_completion_from_message(messages).strip()

    return response

# Tambahkan kolom 'sentimen' berdasarkan hasil dari fungsi sentiment_single
df_testing['one-shot_pos'] = df_testing['text_1'].apply(sentiment_single)

# Cetak dataframe dengan kolom sentimen yang baru
print(df_testing)


## One-Shot Negative

In [ ]:
import pandas as pd
from openai import OpenAI

def get_completion_from_message(message,
                                model="gpt-4o",
                                temperature=0):
    client = OpenAI(
        api_key='OPENAI_API_KEY',
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

def sentiment_single(ulasan):
    delimiter = "#####"

    # Menyediakan contoh one-shot negatif
    example_input = "5 bintang kalo easy withdraw"
    example_output = "negatif"

    system_message = f"""Saya sedang melakukan analisis sentimen terhadap ulasan pelanggan dari sebuah aplikasi \
    exchange crypto bernama tokocrypto.
    Berikut adalah contoh ulasan dan jawabannya:
    Ulasan: {example_input}
    Jawaban: {example_output}

    Petunjuk:
    - Jawab hanya dengan satu kata: "positif" atau "negatif" berdasarkan ulasan yang diberikan.
    - Ulasan: {delimiter}{ulasan}{delimiter}"""

    system_message = {'role': 'system', 'content': system_message}

    # Menyiapkan pesan pengguna untuk ulasan yang ingin diuji
    user_message = [{'role': 'user', 'content': f"{delimiter}{ulasan}{delimiter}"}]

    # Gabungkan pesan sistem dan pesan pengguna
    messages = [system_message] + user_message

    # Dapatkan respons model
    response = get_completion_from_message(messages).strip()

    return response

# Tambahkan kolom 'sentimen' berdasarkan hasil dari fungsi sentiment_single
df_testing['one-shot_neg'] = df_testing['text_1'].apply(sentiment_single)

# Cetak dataframe dengan kolom sentimen yang baru
print(df_testing)


## Few-Shot

In [ ]:
import pandas as pd
from openai import OpenAI

def get_completion_from_message(message,
                                model="gpt-4o",
                                temperature=0):
    client = OpenAI(
        api_key='OPENAI_API_KEY',
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

def sentiment_single(ulasan):
    delimiter = "#####"

    # Menyediakan beberapa contoh (few-shot)
    examples = [
        {"ulasan": "ui aplikasinya simple, jadi bikin gabingung buat orang yang baru belajar", "sentimen": "positif"},
        {"ulasan": "invest mudah kali disini asli wkwkkw", "sentimen": "positif"},
        {"ulasan": "woah aplikasi amanah simple gampang dipahami pemula belajar recommended", "sentimen": "positif"},
        {"ulasan": "kalo konversi saldo kecil ke tko udah bisa lagi, saya kasih bintang tujuh", "sentimen": "negatif"},
        {"ulasan": "butuh widget cepat", "sentimen": "negatif"},
        {"ulasan": "tolong tambahkan fitur persentase keuntungan / kerugian dari rata rata harga coin saat di beli \
        contoh seperti pluang...", "sentimen":"negatif"},
        {"ulasan": "keseluruhan sudah sangat baik prosesnya cepat, tidak ada kendala saat pembelian note untuk konversi \
        koin kecil ke tko tolong dibuka secepatnya", "sentimen":"positif"},
        {"ulasan": "kebalilan fungsi ss!!! fungsi ss penting.", "sentimen":"negatif"},
        {"ulasan": "lebih baik daripada yg sebelah soal stability tapi soal grafik analysis masih kalah lohh", "sentimen":"positif"},
        {"ulasan": "versi ipad enggak responsive oii", "sentimen":"negatif"},
        {"ulasan": "tokocryto sangat bagus untuk bermain jual beli coin", "sentimen":"positif"},
        {"ulasan": "sudah 2 minggu akun belum diverifikasi juga payah", "sentimen":"negatif"},
        {"ulasan": "tokocrypto mudah digunakan karena penggunaan ui nya sederhana", "sentimen":"positif"},
        {"ulasan": "perjuangan banget bisa approved kyc level 1 nya.. udah lama install tpi baru bisa sepenuhnya\
         join...alhamdulillah... smoga bisa lebih cuan cuan cuan lagi disini.. pertanyaan, brapa lama waktu yg \
         dibutuhkan setelah top up terkirim sampai bisa transaksi ya? klo agak lama mungkim bisa dipercepat kah?", "sentimen": "negatif"}
    ]

    # Membuat pesan sistem dengan beberapa contoh
    example_messages = ""
    for example in examples:
        example_messages += f"Ulasan: {example['ulasan']}\nJawaban: {example['sentimen']}\n\n"

    system_message = f"""Saya sedang melakukan analisis sentimen terhadap ulasan pelanggan dari sebuah aplikasi \
    exchange crypto bernama tokocrypto.
    Berikut adalah beberapa contoh ulasan dan jawabannya:
    {example_messages}

    Petunjuk:
    - Jawab hanya dengan satu kata: "positif" atau "negatif" berdasarkan ulasan yang diberikan.
    - Ulasan: {delimiter}{ulasan}{delimiter}"""

    system_message = {'role': 'system', 'content': system_message}

    # Menyiapkan pesan pengguna untuk ulasan yang ingin diuji
    user_message = [{'role': 'user', 'content': f"{delimiter}{ulasan}{delimiter}"}]

    # Gabungkan pesan sistem dan pesan pengguna
    messages = [system_message] + user_message

    # Dapatkan respons model
    response = get_completion_from_message(messages).strip()

    return response

# Tambahkan kolom 'sentimen' berdasarkan hasil dari fungsi sentiment_single
df_testing['few-shot'] = df_testing['text_1'].apply(sentiment_single)

# Cetak dataframe dengan kolom sentimen yang baru
print(df_testing)


## Sentiment Analysis All Data

In [ ]:
def get_completion_from_message(message,
                                model="gpt-4o",
                                temperature=0):
    client = OpenAI(
        api_key='OPENAI_API_KEY',
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

def sentiment(ulasan):
    delimiter = "#####"
    system_message = f"""Saya sedang melakukan analisis sentimen terhadap ulasan pelanggan dari sebuah aplikasi\
     exchange crypto bernama tokocrypto
    Petunjuk:
    Jawab hanya dengan satu kata saja: "positif" atau "negatif"!"""

    system_message = {'role': 'system', 'content': system_message}

    # Menyiapkan pesan pengguna untuk ulasan yang ingin diuji
    user_message = [{'role': 'user', 'content': f"{delimiter}{ulasan}{delimiter}"}]

    # Gabungkan pesan sistem dan pesan pengguna
    messages = [system_message] + user_message

    # Dapatkan respons model
    response = get_completion_from_message(messages).strip()

    return response

# Tambahkan kolom 'sentimen' berdasarkan hasil dari fungsi sentiment
df['zero-shot'] = df['text_1'].apply(sentiment)

# Cetak dataframe dengan kolom sentimen yang baru
print(df)


## BERT

In [ ]:
# Memuat tokenizer pretrained dari model IndoBERT-base-p2 untuk tokenisasi teks berbahasa Indonesia
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
inputs_tokocrypto = tokenizer(df["text_1"].tolist(), return_tensors='tf', truncation=True, padding=True, max_length=128)

In [ ]:
def encode_text_bert(input_tokens, data_name, batch_size=128, load=False):
    model = TFAutoModel.from_pretrained('indobenchmark/indobert-base-p2')

    if not load:
        all_outputs = []
        for i in tqdm(range(0, len(input_tokens['input_ids']), batch_size)):
            batch = {k: v[i:i+batch_size] for k, v in input_tokens.items()}
            outputs = model(batch, output_hidden_states=True)

            outputs = tf.reduce_mean(outputs.last_hidden_state, axis=1)
            all_outputs.append(outputs)

        final_output = tf.concat(all_outputs, axis=0)
        SAVE_DIR = f'{data_name}_BERT.pkl'
        joblib.dump(final_output, SAVE_DIR)
        return final_output

    else:
        SAVE_DIR = f'{data_name}_BERT.pkl'
        emb = joblib.load(SAVE_DIR)
        return emb

In [ ]:
emb = encode_text_bert(inputs_tokocrypto, 'tokocrypto')

## c-TFIDF

In [ ]:
# Class Based TFIDF
def tf_idf_count(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range).fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_words_per_topic(tf_idf, count, docs_per_topic, n = 10):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    tf_idf_top_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    top_words = []
    for i in range(len(labels)):
      top_words.append([words[j] for j in indices [i]][::-1])
    return top_words, tf_idf_top_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['topic'])
                     .text
                     .count()
                     .reset_index()
                     .rename({"topic": "topic", "text": "size"}, axis='columns')
                     .sort_values("size", ascending=False))
    return topic_sizes

def c_tf_idf(docs_per_topic, m, n_words=10, n_gram_range=(1,1)):
    tf_idf, count = tf_idf_count(docs_per_topic.text.values, m=m)
    top_words, tf_idf_top_words = extract_top_words_per_topic(tf_idf, count, docs_per_topic, n_words)

    return tf_idf, count, top_words, tf_idf_top_words

## BERT EFCM c-TFIDF

In [ ]:
def BERT_EFCM_cTFIDF(df, data_name, n_clusters, m=1.1, n_words=10,
                     n_gram_range=(1,1)):

    # BERT
    print('Creating Text Embedding using BERT...')
    emb = encode_text_bert(df, data_name='tokocrypto', load=True)

    # EFCM
    print('Training EFCM...')

    svd = TruncatedSVD(n_components = 5)
    emb_tsvd = svd.fit_transform(emb)

    initkm = KMeans(n_clusters=n_clusters, n_init=1).fit(emb_tsvd)
    cntr, u = fcmeans(emb_tsvd.T, n_clusters, m, error=0.0001, maxiter=200, init=initkm.cluster_centers_.T)
    cluster_membership = np.argmax(u, axis=0)

    # cTFIDF
    print('Extracting top words using cTFIDF')
    docs_df = df[['text_2']].copy()
    docs_df.rename({'text_2': 'text'}, axis=1, inplace=True)

    docs_df['topic'] = cluster_membership.reshape(-1,1)

    docs_df['doc_id'] = range(len(docs_df))

    docs_per_topic = docs_df.dropna(subset=['text']).groupby(['topic'], as_index = False).agg({'text': ' '.join})

    c_tf_idf_, count, top_words, top_words_c_tf_idf = c_tf_idf(docs_per_topic, docs_df.shape[0],
                                                              n_words=n_words, n_gram_range=n_gram_range)

    print('FINISHED')

    return emb, emb_tsvd, cluster_membership, c_tf_idf_, count, top_words, top_words_c_tf_idf

## TC-W2V

In [ ]:
#Nilai Coherence
### Memuat Model Word2Vec
w2v_model = gensim.models.Word2Vec.load("/content/drive/MyDrive/Deep Learning/Data/word2vec/idwiki-berita/w2v-model.bin")

def calculate_coherence(w2v_model, term_rankings):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for i in range(len(term_rankings[topic_index])):
            for j in range(i + 1, len(term_rankings[topic_index])):
                term_i = term_rankings[topic_index][i]
                term_j = term_rankings[topic_index][j]
                if term_i in w2v_model.wv.key_to_index and term_j in w2v_model.wv.key_to_index:
                    pair_scores.append(w2v_model.wv.similarity(term_i, term_j))

        # get the mean for all pairs in this topic
        if pair_scores:
            topic_score = sum(pair_scores) / len(pair_scores)
            overall_coherence += topic_score

    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

## EFCM

In [ ]:
# Membuat looping untuk parameter m dari 1.05 sampai 3.00 dengan interval 0.05
for m in np.arange(1.05, 3.00, 0.05):
    print('=' * 100)
    print('Fuzziness:', round(m, 2))

    tc_w2v = []
    best_tc_w2v = 0
    all_tc_w2v_means = []

    for n_topic in range(1, 11):
        print('=' * 100)
        print('TOPIC:', n_topic)

        tc_w2v_sim = []
        best_tc_w2v_sim = 0

        for sim in range(2):
            print('=' * 100)
            print('SIMULATION:', sim + 1)
            print('=' * 100)

            # Memanggil fungsi BERT_EFCM_cTFIDF function
            emb_, emb_tsvd_, cluster_member_, c_tf_idf_, count_, top_words_, top_words_c_tf_idf_ = BERT_EFCM_cTFIDF(
                df,
                m=round(m, 2),
                data_name='tokocrypto',
                n_clusters=n_topic
            )

            # Menghitung nilai coherence
            coherence = calculate_coherence(w2v_model, top_words_)
            tc_w2v_sim.append(coherence)

            if best_tc_w2v_sim < coherence:
                best_tc_w2v_sim = coherence
                best_emb_tsvd_sim, best_cluster_member_sim, best_c_tf_idf_sim, best_count_sim, best_top_words_sim, \
                best_top_words_ctfidf_sim = (
                    emb_tsvd_, cluster_member_, c_tf_idf_, count_, top_words_, top_words_c_tf_idf_
                )

            print('=' * 100)
            print(f'FINISHED; {n_topic} TOPICS; SIMULATION {sim + 1}; TC-W2V {round(tc_w2v_sim[-1], 3)}')
            print(f'BEST CURRENT TC-W2V: {best_tc_w2v_sim}; TOP WORDS: {best_top_words_sim}')

        tc_w2v.append(tc_w2v_sim)

        mean_tc_w2v = np.array(tc_w2v_sim).mean()
        all_tc_w2v_means.append(mean_tc_w2v)

        if best_tc_w2v < mean_tc_w2v:
            best_tc_w2v = mean_tc_w2v
            best_emb_tsvd, best_cluster_member, best_c_tf_idf, best_count, best_top_words, best_top_words_ctfidf = (
                best_emb_tsvd_sim, best_cluster_member_sim, best_c_tf_idf_sim, best_count_sim, best_top_words_sim,\
                best_top_words_ctfidf_sim
            )
            best_n_topic = n_topic

        print('-' * 100)
        print(f'FINISHED; {n_topic} TOPICS ALL SIMULATION; TC-W2V mean {mean_tc_w2v}')
        print(f'BEST N TOPICS: {best_n_topic}; BEST TC-W2V: {best_tc_w2v}; BEST TOP WORDS: {best_top_words}')
        print('-' * 100)

    # Membuat grafik
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, 11), all_tc_w2v_means, marker='o', label=f'm={round(m, 2)}')
    plt.title(f'Topic Coherence vs Number of Topics (m={round(m, 2)})')
    plt.xlabel('Number of Topics')
    plt.ylabel('Mean TC-W2V')
    plt.xticks(range(1, 11))
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
# Menyimpan parameter m yang dipilih
best_m = 2.35
best_n_topic = best_n_topic

print('=' * 100)
print(f'Applying BERT_EFCM_cTFIDF with Best Parameters: m={round(best_m, 2)}, n_clusters={best_n_topic}')

emb, emb_tsvd, cluster_membership, c_tf_idf_, count, top_words, top_words_c_tf_idf = BERT_EFCM_cTFIDF(
    df,
    data_name='tokocrypto',
    n_clusters=best_n_topic,
    m=round(best_m, 2)
)

print('BERT_EFCM_cTFIDF applied successfully.')

In [ ]:
# Melakukan cluster pada dataset
results_df = df.copy()
results_df['topics'] = cluster_membership

results_df[['text_1','text_2', 'topics']]

In [ ]:
# Menghitung jumlah ulasan untuk setiap topik (cluster)
topic_counts = results_df['topics'].value_counts()
print(topic_counts)

## Top Words

In [ ]:
# Membuat dictionary untuk menyimpan kata-kata teratas dari setiap topik
topics_words = {}

# Looping untuk setiap topik (misalnya, ada 3 topik dalam top_words_c_tf_idf)
for i in range(len(top_words_c_tf_idf)):
    # Menyimpan kata-kata teratas dari setiap topik ke dalam dictionary
    topics_words[f"Topic {i}"] = ', '.join(word[0] for word in top_words_c_tf_idf[i])

# Menampilkan hasil
for topic, words in topics_words.items():
    print(f"{topic}: {words}")

## Dokumen Representatif

In [ ]:
def get_topic_vectors(ctfidf_matrix, topic_words, word_index):
    # Jumlah topik
    num_topics = ctfidf_matrix.shape[1]

    vocab_size = ctfidf_matrix.shape[0]

    topic_vectors = np.zeros((num_topics, vocab_size))

    for topic_idx, words in enumerate(topic_words):
        for word in words:
            if word in word_index:
                word_idx = word_index[word]
                topic_vectors[topic_idx, word_idx] = ctfidf_matrix[word_idx, topic_idx]

    return topic_vectors

topic_vectors = get_topic_vectors(best_c_tf_idf, best_top_words, best_count.vocabulary_)

In [ ]:
def cosine_similarity(a, b):
    dot_product = np.dot(b, a.T)

    norm_b = np.linalg.norm(b, axis=1)
    norm_a = np.linalg.norm(a)

    if norm_a == 0 or np.any(norm_b == 0):
        return np.array([0] * b.shape[0])

    cosine_similarity = dot_product.ravel() / (norm_b * norm_a)

    return cosine_similarity

In [ ]:
results_df['text_2'] = results_df['text_2'].astype(str)
sim_scores = []
for i in tqdm(range(results_df.shape[0])):
    docs_vector = np.zeros(best_c_tf_idf.shape[0])
    for word in results_df['text_2'].values[i].split():
            if word in best_count.vocabulary_:
                word_idx = best_count.vocabulary_[word]
                topic_idx = results_df['topics'].values[i]
                docs_vector[word_idx] = best_c_tf_idf[word_idx, topic_idx]

    sim_scores.append(cosine_similarity(topic_vectors[topic_idx].reshape(1,-1), docs_vector.reshape(1,-1))[0])

results_df['scores'] = sim_scores
results_df

In [ ]:
def print_top_documents_by_topic(df, num_top_docs=5):
    unique_topics = sorted(df['topics'].unique())

    for topic in unique_topics:
        print(f"Topics: {topic}")
        print("-" * 50)

        top_docs = df[df['topics'] == topic].nlargest(num_top_docs, 'scores')

        for index, row in top_docs.iterrows():
            print(f"{index + 1}. {row['text_1']} - Score: {row['scores']:.6f}")

        print("=" * 50)

print_top_documents_by_topic(results_df, num_top_docs=6)

## Interpretasi Topik dengan GPT

In [ ]:
# Inisialisasi OpenAI API
openai.api_key = "OPENAI_API_KEY"

# Jumlah topik
num_topics = len(top_words_c_tf_idf)

# Loop untuk setiap topik dan membuat prompt untuk GPT
for topic_num in range(num_topics):
    # Mendapatkan kata-kata teratas untuk topik saat ini
    topic_words = topics_words[f"Topic {topic_num}"]

    # Mendapatkan dokumen teratas untuk topik saat ini
    representative_docs = "\n".join([f"{i+1}. {doc}" \
                                     for i, doc in enumerate(top_docs_per_topic[f"Topic {topic_num}"])])

    # Membuat prompt
    message = f"""Tentukan interpretasi topik dari {topic_words} dengan mengacu pada dokumen {representative_docs},\
     Tuliskan interpretasi dalam satu kalimat singkat (maksimal 15 kata), dan menggunakan bahasa Indonesia."""

    # Membuat permintaan ke OpenAI GPT
    completion = openai.chat.completions.create(
        model="gpt-4o",
        seed=42,
        messages=[{"role": "user", "content": message}]
    )

    # Menampilkan hasil interpretasi dari GPT
    label = completion.choices[0].message.content
    print(f"Interpretasi untuk Topic {topic_num}: {label}")
    print("=" * 50)
